In [ ]:
import pandas as pd
import numpy as np
import phonenumbers
import re


coluna_telefone = input(" coluna de telefone: ")

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', None)




def validar_telefone(numero, codigo_pais='MX'):
    """Valida e normaliza números de telefone"""
    if pd.isna(numero) or numero is None:
        return None
    numero = re.sub(r'\D', '', str(numero)).strip()
    if not numero:
        return None
    try:
        parsed = phonenumbers.parse(numero, codigo_pais)
        if phonenumbers.is_valid_number(parsed):
            return numero
    except phonenumbers.NumberParseException:
        pass
    return None



def verificar_telefones(df, coluna_telefone):
    """Função principal que executa todo o processo de verificação"""
    # Carregamento de dados
    print("🔄 Carregando CSVs...")
    ladas_mexico = pd.read_csv('./csv/ladas_mexico.csv')

    # Normalização
    print("🧹 Normalizando campos de cidade e estado...")
    for df_aux in [ladas_mexico]:
        df_aux["estado"] = df_aux["estado"].astype(str).str.strip().str.lower()
        df_aux["ciudad"] = df_aux["ciudad"].astype(str).str.strip().str.lower()

    # Múltiplos DDDs por cidade + estado
    print("🔍 Identificando cidades com múltiplas LADAs...")
    claves_por_par = (
        ladas_mexico
        .groupby(['ciudad', 'estado'])['lada'].nunique().reset_index().query('lada > 1')
        .merge(
            ladas_mexico.groupby(['ciudad', 'estado'])['lada'].unique().reset_index().rename(columns={'lada': 'claves_lada'}),
            on=['ciudad', 'estado']
        )[['ciudad', 'estado', 'claves_lada']]
    )
    print(f"📌 {len(claves_por_par)} combinações cidade+estado com múltiplas LADAs.")

  



    def processar_linha(row):
        telefone = row.get(coluna_telefone)
        estado = str(row.get("n_entidad", "")).strip().lower()
        cidade = str(row.get("city", "")).strip().lower()
        resultados = {}

        if isinstance(telefone, str) and len(telefone) in (7, 8):
            mask_cp = (claves_por_par["estado"] == estado) & (claves_por_par["ciudad"] == cidade)
            rec_cp = claves_por_par[mask_cp]

            if not rec_cp.empty:
                ddds = rec_cp.iloc[0]["claves_lada"]
                for i, ddd in enumerate(ddds):
                    numero = str(ddd) + telefone
                    validado = validar_telefone(numero)
                    if validado:
                        col = "tel_ddd_adicionado" if i == 0 else f"tel_ddd_adicionado_{i+1}"
                        resultados[col] = validado
                return pd.Series(resultados)

            mask_lm = (ladas_mexico["estado"] == estado) & (ladas_mexico["ciudad"] == cidade)
            rec_lm = ladas_mexico[mask_lm]

            if not rec_lm.empty:
                ddd = rec_lm.iloc[0]["lada"]
                numero = str(ddd) + telefone
                validado = validar_telefone(numero)
                if validado:
                    resultados["tel_ddd_adicionado"] = validado

        return pd.Series(resultados)

  
  



🔄 Carregando CSVs...
🧹 Normalizando campos de cidade e estado...
🔍 Identificando cidades com múltiplas LADAs...
📌 40 combinações cidade+estado com múltiplas LADAs.
🛠️ Reconstruindo números com DDD...
✅ Limpando e validando campo original 'telefone_scraping'...

📞 Números válidos em 'telefone_scraping': 0
📞 Números reconstruídos com DDDs: 0.0
📚 Colunas 'tel_ddd_adicionado' criadas: 0
✅ Total de números válidos encontrados: 0.0
💾 Salvando CSV final...


In [5]:

import pandas as pd
import numpy as np
import phonenumbers
import re


def validar_telefone(numero, codigo_pais='MX'):
    """Valida e normaliza números de telefone"""
    if pd.isna(numero) or numero is None:
        return None
    numero = re.sub(r'\D', '', str(numero)).strip()
    if not numero:
        return None
    try:
        parsed = phonenumbers.parse(numero, codigo_pais)
        if phonenumbers.is_valid_number(parsed):
            return numero
    except phonenumbers.NumberParseException:
        pass
    return None



print(validar_telefone("5555582690"))

5555582690
